In [1]:
import tensorflow as tf, pkgutil, sys
print("PY:", sys.executable)
installed = pkgutil.find_loader("tensorflow") is not None
print("TF installed? ", installed)
if installed:
    print("TF version:", tf.__version__)
    print("GPUs:", tf.config.list_physical_devices('GPU'))


/Users/tingyiwei/miniforge3/envs/gnn_env/lib/python3.9/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/tingyiwei/miniforge3/envs/gnn_env/lib/python3.9/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/tingyiwei/miniforge3/envs/gnn_env/lib/python3.9/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_

: 

## 注释1
TensorFlow ↔ protobuf 版本冲突 导致在 导入 TF 时直接崩了（libc++abi ... system_error: mutex lock failed）。
前面那串 “gencode 5.28.3 与 runtime 6.31.1 不匹配” 警告已揭示关键：你当前装的 protobuf 是 6.x，而 TF 这套 wheel 用的是 5.x 生成的代码，结果触发底层 C++ 绑定崩溃。

## 注释2
protobuf = Google 的二进制“翻译官”。
它帮 TensorFlow、gRPC、NLP 模型等在「Python ↔ C++」或「本地 ↔ 远程」之间高效传递结构化数据。

In [ ]:
# 卸载冲突的版本
python -m pip uninstall -y tensorflow tensorflow-macos tensorflow-metal keras keras-nightly protobuf

In [ ]:
## 安装不冲突的版本
# 建议也把 numpy 钉到 <2，避免个别二进制不兼容
python -m pip install "numpy<2" "protobuf==3.20.3"
python -m pip install "tensorflow-macos==2.15.1" "tensorflow-metal==1.1.0"

In [3]:
%pip install --no-cache-dir --force-reinstall "numpy<2" "protobuf==3.20.3" \
  "tensorflow-macos==2.15.1" "tensorflow-metal==1.1.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 3.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: wheel
    Found existing installation: wheel 0.45.1
    Uninstalling wheel-0.45.1:
      Successfully uninstalled wheel-0.45.1
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six-1.17.0━━━━━━━━━━━━━━━━━━━━━━━━━ 1/6 [six]
  Attempting uninstall: protobuf━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/6 [six]
    Found existing installation: protobuf 3.20.3━━━━━━━━━━━━━━ 1/6 [six]
    Uninstalling protobuf-3.20.3:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/6 [six]
      Successfully uninstalled protobuf-3.20.3━━━━━━━━━━━━━━━━━━━━ 2/6 [protobuf]
  Attempting uninstall: numpym╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/6 [protobuf]
    Found existing installation: numpy 1.26.4━━━━━━━━━━━━━━━━━ 2/6 [protobuf]
    Uninstalling numpy-1.26.4:m━━━━━━━━━━━━━━━

In [2]:
import sys, importlib.util
print("PY:", sys.executable)
print("TF spec:", importlib.util.find_spec("tensorflow"))

import tensorflow as tf
print("TF:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))


PY: /Users/tingyiwei/miniforge3/envs/gnn_env/bin/python
TF spec: ModuleSpec(name='tensorflow', loader=<_frozen_importlib_external.SourceFileLoader object at 0x1062a9cd0>, origin='/Users/tingyiwei/miniforge3/envs/gnn_env/lib/python3.9/site-packages/tensorflow/__init__.py', submodule_search_locations=['/Users/tingyiwei/miniforge3/envs/gnn_env/lib/python3.9/site-packages/tensorflow', '/Users/tingyiwei/miniforge3/envs/gnn_env/lib/python3.9/site-packages/tensorflow/_api/v2'])
TF: 2.15.1
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# 虽然装了 tensorflow-macos，但真正的顶层包 tensorflow 没被装上，所以 import tensorflow 一直是 ModuleNotFoundError
证明：
python -m pip show -f tensorflow-macos
Files:
  tensorflow_macos-2.15.1.dist-info/...
  .../top_level.txt
只有一个“元包”的 dist-info，没有 tensorflow/__init__.py 等实际模块文件；因此 Python 找不到 tensorflow。

## 原因： 
tensorflow-macos 只是一个 meta 包（壳包），按理会拉取真正的 tensorflow。在某些环境/约束组合下（比如我们先固定了 numpy<2、protobuf==3.20.3 等），依赖解析没把 tensorflow 本体拉下来，就出现了“看似已装、实际没模块”的现象。
-> 那还不能先固定版本 好麻烦啊啊啊啊啊

In [ ]:
conda activate gnn_env

# 1) 先卸掉这个“壳包”，避免干扰（可选但建议）
python -m pip uninstall -y tensorflow-macos

# 2) 直接装真正的本体（2.15.1 与我们当前 numpy/protobuf 组合最稳）
python -m pip install --no-cache-dir --force-reinstall "tensorflow==2.15.1"

# 3) 继续确保 GPU 后端在（如果被卸了就再装回）
python -m pip install --no-cache-dir --force-reinstall "tensorflow-metal==1.1.0"

# 4) 终端里先验证
python - << 'PY'
import sys, importlib.util
print("PY:", sys.executable)
print("TF spec:", importlib.util.find_spec("tensorflow"))
import tensorflow as tf
print("TF:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))
PY


In [1]:
## 安装成功！！

import tensorflow as tf, sys
print("PY:", sys.executable)
print("TF:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))

# 简单触发一次计算
x = tf.random.normal((2048, 2048))
y = tf.random.normal((2048, 2048))
z = tf.matmul(x, y)
z.numpy().shape


PY: /Users/tingyiwei/miniforge3/envs/gnn_env/bin/python
TF: 2.15.1
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-10-14 17:23:15.600815: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2025-10-14 17:23:15.600831: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2025-10-14 17:23:15.600837: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 25.92 GB
2025-10-14 17:23:15.600871: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-10-14 17:23:15.600892: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


(2048, 2048)

In [3]:
# alfabet验证
import alfabet_lite as al
al.__file__


'/Users/tingyiwei/miniforge3/envs/gnn_env/lib/python3.9/site-packages/alfabet_lite/__init__.py'

In [4]:
from alfabet_lite.fragment import canonicalize_smiles
s = "CCO"   # 例子：乙醇
print("canon:", canonicalize_smiles(s))


ImportError: cannot import name 'canonicalize_smiles' from 'alfabet_lite.fragment' (/Users/tingyiwei/miniforge3/envs/gnn_env/lib/python3.9/site-packages/alfabet_lite/fragment.py)

In [5]:
# alfabet-lite里有啥
import alfabet_lite.fragment as frag
print(dir(frag))


['Counter', 'Dict', 'Iterator', 'Molecule', 'RDLogger', 'Type', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'check_stereocenters', 'count_atom_types', 'count_stereocenters', 'fragment_iterator', 'get_bond_type', 'get_fragments', 'logging', 'pd', 'rdkit']
